# Leukamia using Clest

In [1]:
# Install curatedTCGAData if not already installed
# if (!require("BiocManager")) install.packages("BiocManager")
# BiocManager::install("curatedTCGAData")
# Load library
library("curatedTCGAData")
library("SummarizedExperiment")

options(scipen=10)
options(repr.plot.width=12, repr.plot.height=12)

Cargando paquete requerido: MultiAssayExperiment

Cargando paquete requerido: SummarizedExperiment

Cargando paquete requerido: MatrixGenerics

Cargando paquete requerido: matrixStats


Adjuntando el paquete: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, r

In [2]:
# Download BRCA RNA-Seq data
brca_data <- curatedTCGAData(diseaseCode = "BRCA", 
                             assays = "RNASeq2GeneNorm", 
                             version = "2.1.1",  # Specify the version
                             dry.run = FALSE)  # Set dry.run = TRUE to preview first


Querying and downloading: BRCA_RNASeq2GeneNorm-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Querying and downloading: BRCA_colData-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Querying and downloading: BRCA_sampleMap-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

Querying and downloading: BRCA_metadata-20160128

see ?curatedTCGAData and browseVignettes('curatedTCGAData') for documentation

loading from cache

harmonizing input:
  removing 14373 sampleMap rows not in names(experiments)
  removing 5 colData rownames not in sampleMap 'primary'



In [3]:
experiments(brca_data)

ExperimentList class object of length 1:
 [1] BRCA_RNASeq2GeneNorm-20160128: SummarizedExperiment with 18300 rows and 1212 columns

In [6]:
# Extract full sample IDs and short patient IDs
sample_ids <- colnames(expression)  # Full TCGA sample IDs (e.g., "TCGA-3C-AAAU-01A-11R")
sample_ids_short <- substr(sample_ids, 1, 12)  # Short IDs (e.g., "TCGA-3C-AAAU")
metadata_patient_ids <- metadata$patientID  # Short patient IDs from metadata

In [7]:
# Find common patients
common_samples_short <- intersect(sample_ids_short, metadata_patient_ids)
# Create a mapping table from full IDs to short IDs
sample_map <- data.frame(full_id = sample_ids, short_id = sample_ids_short)
# Get full sample IDs corresponding to common short IDs
common_full_ids <- sample_map$full_id[sample_map$short_id %in% common_samples_short]

In [9]:
# Subset metadata to only include common patients
metadata_subset <- metadata[metadata$patientID %in% common_samples_short, ]
pam50_labels <- metadata_subset$PAM50.mRNA

In [11]:
# Removing NA PAM50 labels before getting top 200 individuasl by RNA variance
metadata_subset_filtered <- metadata_subset[!is.na(metadata_subset$PAM50.mRNA), ]

# Get the list of valid sample IDs after removing NA PAM50
valid_samples <- sample_map$full_id[sample_map$short_id %in% metadata_subset_filtered$patientID]
# Subset expression data to only include samples with valid PAM50 labels
expression_filtered <- expression[, valid_samples]

In [18]:
# Identify top 200 variance genes
gene_variances <- apply(expression_filtered, 1, var)
top_200_genes <- names(sort(gene_variances, decreasing = TRUE))[1:200]
top_200_expression <- expression_filtered[top_200_genes, ]

# Transpose expression matrix: samples as rows, genes as columns
top_200_df <- as.data.frame(t(top_200_expression))

# Add "description" column with **full sample IDs**
top_200_df$description <- rownames(top_200_df)

# Extract patient ID from sample ID (first 12 characters)
top_200_df$patient_short_id <- substr(top_200_df$description, 1, 12)

In [19]:
# Match and add PAM50 labels **with correctly filtered metadata**
top_200_df$PAM50 <- metadata_subset_filtered$PAM50.mRNA[match(
    top_200_df$patient_short_id, 
    metadata_subset_filtered$patientID
)]
# Remove temporary patient ID column
top_200_df$patient_short_id <- NULL

# Moving "description" and "PAM50" to the beginning
top_200_df <- top_200_df[, c("description", "PAM50", setdiff(names(top_200_df), c("description", "PAM50")))]

In [22]:
head(top_200_df)

,description,PAM50,LTF,NPY1R,PGR,RPS28,SFRP1,PLIN4,S100P,PDZK1,⋯,MMP9,SH3RF2,PADI2,GPAT2,PDE3B,FGFR3,GALNT14,CYP21A2,WISP2,PDE8B
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-A1-A0SD-01,TCGA-A1-A0SD-01,Luminal A,11.978259,12.778363,9.485349,8.201800,10.709060,10.383206,9.647907,7.358335,⋯,9.400612,6.919838,8.805018,5.132059,6.726067,10.147290,7.240188,8.270969,7.065917,8.211493
TCGA-A1-A0SE-01,TCGA-A1-A0SE-01,Luminal A,12.779078,8.741927,12.807370,3.644445,13.078639,6.667984,5.943417,9.736929,⋯,6.754074,7.920084,10.368784,4.350716,5.254503,9.319850,5.339038,7.419833,8.312402,6.309411
TCGA-A1-A0SH-01,TCGA-A1-A0SH-01,Luminal A,11.231554,5.948533,12.176581,8.229385,11.346664,5.115645,6.905720,2.115633,⋯,9.890979,5.769084,9.537708,6.394867,5.054241,4.145392,4.085272,4.433721,10.229385,7.410264
TCGA-A1-A0SJ-01,TCGA-A1-A0SJ-01,Luminal A,7.804147,9.461259,11.884642,4.163829,10.991285,11.145075,9.916533,8.783936,⋯,9.078549,5.040971,7.239116,6.064818,8.942509,8.637907,6.080500,8.568362,10.756743,6.263365
TCGA-A1-A0SK-01,TCGA-A1-A0SK-01,Basal-like,4.089040,5.483900,2.793584,9.945638,12.763559,1.141498,4.141506,3.463439,⋯,9.403602,2.582074,11.992810,1.656084,8.225936,8.133434,10.766694,1.809250,2.978013,5.154332
TCGA-A1-A0SM-01,TCGA-A1-A0SM-01,Luminal B,7.826276,4.526250,6.455368,9.440371,8.159795,3.336426,11.191101,7.383551,⋯,9.542832,7.889097,9.475339,5.565247,4.762694,6.574833,4.584355,10.308822,9.321558,4.693978


In [24]:
save(top_200_df, file = "200TCGAData.RData")